In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date

# pass headers in a dict to the headers parameter to mimic a real human doing the request using a web browser
headers = {
    'accept': '*/*',
    'content-type': 'text/html;charset=UTF-8',
    'content-encoding': 'gzip',
    'origin': 'www.zillow.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
}

In [2]:
# two paranmeters that you can change
num_listings = 150

rental = False

# initiators listed here
address = []
price = []
h_link = []
timestamp = []
img = []
zestimate = []
street_address = []
num_rooms = []
num_bedrooms = []
num_bathrooms = []
town = []
state = []
postalcode = []
year_built = []
days_on_zillow = []
number_of_view = []
sqft = []

# record when the data is crawled
date_today = date.today()

count = 1
# request link
if rental == True:
    url = 'https://www.zillow.com/manhattan-new-york-ny-10023/{}_p/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%2210023%22%2C%22mapBounds%22%3A%7B%22west%22%3A-74.00444063244629%2C%22east%22%3A-73.95895036755371%2C%22south%22%3A40.76165781702343%2C%22north%22%3A40.79194600282301%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A61637%2C%22regionType%22%3A7%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D'.format(count)
else:
    url = 'https://www.zillow.com/manhattan-new-york-ny-10023/{}_p/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%2210023%22%2C%22mapBounds%22%3A%7B%22west%22%3A-74.00444063244629%2C%22east%22%3A-73.95895036755371%2C%22south%22%3A40.76165781702343%2C%22north%22%3A40.79194600282301%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A61637%2C%22regionType%22%3A7%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D'.format(count)
    
# open the url and get a response
r = requests.get(url, headers=headers)

# read the response content with soup
soup = BeautifulSoup(r.content, 'html.parser')

# number of listing crawled
crawled_listing = 0

while num_listings > crawled_listing:
            
    for s1 in soup.find_all('div', {'class':"list-card-info"}):
        
        # sqft
        try:
            sqft.append(s1.findAll('ul', {'class':'list-card-details'})[0].contents[2].contents[0])
        except Exception:
            sqft.append('N/A')
        
        # address
        try:
            address.append(s1.find('address', {'class':'list-card-addr'}).text)
        except Exception:
            address.append('N/A')
            
        # rental/sell price
        try:
            price.append(s1.find('div', {'class':"list-card-price"}).text)
        except Exception:
            price.append('N/A')
            
        # rental/sell link
        try:
            h_link.append(s1.find('a', {'class':"list-card-link list-card-link-top-margin"}).get('href'))
        except Exception:
            h_link.append('N/A')
            
        # num of bathrooms
        try:
            num_bathrooms.append(s1.findAll('ul', {'class' : 'list-card-details'})[0].contents[1].contents[0])
        except Exception:
            num_bathrooms.append("N/A")
        
        # num of bedrooms
        try:
            num_bedrooms.append(s1.findAll('ul', {'class' : 'list-card-details'})[0].contents[0].contents[0])
        except Exception:
            num_bedrooms.append('N/A')
            
            
        ##############
        ##House Page##
        ##############
        if h_link[-1] != 'N/A':
            try:
                r1 = requests.get(h_link[-1], headers=headers)
            except Exception:
                img.append('N/A')
                zestimate.append('N/A')
                street_address.append('N/A')
                num_rooms.append('N/A')
                town.append('N/A')
                state.append('N/A')
                postalcode.append('N/A')
                year_built.append('N/A')
                days_on_zillow.append('N/A')
                
            s2 = BeautifulSoup(r1.content, 'html.parser')
            link2 = s2.find('div', {'id':'home-details-content'})
            link3 = s2.find('ul',{'class':"zsg-tooltip-viewport"})
            
            #number of view
            try:
                number_of_view.append(s2.findAll('dl', {'class':"hdp__sc-7d6bsa-0 kTZMwK"})[0].contents[4])
            except:
                number_of_view.append("N/A")  
            
            #Days on Zilloz
            try:
                days_on_zillow.append(s2.findAll('dl', {'class':"hdp__sc-7d6bsa-0 kTZMwK"})[0].contents[1])
            except:
                days_on_zillow.append("N/A")   
            
            #Year Built
            try:
                year_built.append(link3.findAll('li', {'class':"hdp__sc-1esuh59-0 dmmleM"})[1].contents[2].contents[0])
            except:
                year_built.append("N/A")            
            
            try:
            # json
                js = str(link2.find('script', {"type":"application/ld+json"})).replace('<script type="application/ld+json">','')
            except Exception:
                street_address.append('N/A')
                num_rooms.append('N/A')
                num_bedrooms.apppend('N/A')
                num_bathrooms.apppend('N/A')
                town.append('N/A')
                state.append('N/A')
                postalcode.append('N/A')
                
            js.replace('</script>','')
            
            data = json.loads(js.replace('</script>',''))

            # street address
            try:
                street_address.append(data['address']['streetAddress'])
            except KeyError:
                street_address.append("N/A")

            # num of rooms
            try:
                num_rooms.append(data['numberOfRooms'])
            except KeyError:
                num_rooms.append("N/A")

            # town
            try:
                town.append(data['address']['addressLocality'])
            except KeyError:
                town.append("N/A")

            # state
            try:
                state.append(data['address']['addressRegion'])
            except KeyError:
                state.append("N/A")

            # postalCode
            try:
                postalcode.append(data['address']['postalCode'])
            except KeyError:
                postalcode.append("N/A")
                
            
        else:
            img.append('N/A')
            zestimate.append('N/A')
            street_address.append('N/A')
            num_rooms.append('N/A')
            town.append('N/A')
            state.append('N/A')
            postalcode.append('N/A')
            year_built.append('N/A')
            days_on_zillow.append('N/A')
            
        
        # timestamp
        timestamp.append(date_today)

        crawled_listing += 1

        # crawled number of listings, break out of loop
        if num_listings == crawled_listing:
            break

    # update request link 
    count += 1
    if rental == True:
        url = 'https://www.zillow.com/manhattan-new-york-ny-10023/{}_p/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%2210023%22%2C%22mapBounds%22%3A%7B%22west%22%3A-74.00444063244629%2C%22east%22%3A-73.95895036755371%2C%22south%22%3A40.76165781702343%2C%22north%22%3A40.79194600282301%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A61637%2C%22regionType%22%3A7%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D'.format(count)
    else:
        url = 'https://www.zillow.com/manhattan-new-york-ny-10023/{}_p/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%2210023%22%2C%22mapBounds%22%3A%7B%22west%22%3A-74.00444063244629%2C%22east%22%3A-73.95895036755371%2C%22south%22%3A40.76165781702343%2C%22north%22%3A40.79194600282301%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A61637%2C%22regionType%22%3A7%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D'.format(count)
 
    
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
# create a dataframe based on the lists created above
df = pd.DataFrame(list(zip(h_link, price, timestamp, street_address, sqft, num_rooms, num_bedrooms, num_bathrooms, town,
                           state, postalcode, year_built, days_on_zillow, number_of_view)),
                  columns = ['Link', 'Price', 'TimeStamp', 'Street Address', 'Square footage', 'Num of Rooms', 'Num of Bedrooms', 'Num of Bathrooms', 'Town', 'State', 'Postal Code', 'Year Built', 'Days on Zillow', 'Number of view'])

In [4]:
df

,Link,Price,TimeStamp,Street Address,Square footage,Num of Rooms,Num of Bedrooms,Num of Bathrooms,Town,State,Postal Code,Year Built,Days on Zillow,Number of view
0,https://www.zillow.com/homedetails/150-W-End-A...,"$685,000",2022-02-15,150 W End Ave APT 2G,754,1,1,1,New York,NY,10023,N/A,N/A,N/A
1,https://www.zillow.com/homedetails/205-W-End-A...,"$725,000",2022-02-15,205 W End Ave APT 24G,--,0,Studio,1,New York,NY,10023,N/A,N/A,N/A
2,https://www.zillow.com/homedetails/3-Riverside...,"$22,000,000",2022-02-15,3 Riverside Dr,"19,251",9,9,11,New York,NY,10023,N/A,N/A,N/A
3,https://www.zillow.com/homedetails/32-W-76th-S...,"$25,975,000",2022-02-15,32 W 76th St,"10,635",6,6,9,New York,NY,10023,N/A,N/A,N/A
4,https://www.zillow.com/homedetails/171-W-73rd-...,"$1,299,000",2022-02-15,171 W 73rd St APT 2,--,2,2,2,New York,NY,10023,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.zillow.com/homedetails/225-W-70th-...,"$768,000",2022-02-15,225 W 70th St APT B,--,1,1,2,New York,NY,10023,N/A,N/A,N/A
131,https://www.zillow.com/homedetails/10-W-66th-S...,"$1,250,000",2022-02-15,10 W 66th St APT 23J,--,1,1,2,New York,NY,10023,N/A,N/A,N/A
132,https://www.zillow.com/homedetails/225-W-60th-...,"$1,750,000",2022-02-15,225 W 60th St APT 9C,"1,181",2,2,2,New York,NY,10023,N/A,N/A,N/A
133,https://www.zillow.com/homedetails/161-W-61st-...,"$1,850,000",2022-02-15,161 W 61st St APT 23F,948,1,1,2,New York,NY,10023,N/A,N/A,N/A


In [5]:
df.to_csv("zillow_listening.csv")